In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [9]:
x_data = obtain_path(
    df=df_rumours, 
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=20
)

x_data.shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:37<00:00, 148.35it/s]


(5568, 20, 384)

Baseline BiLSTM

In [19]:
num_epochs = 100
hidden_dim_sizes = [200]#[100, 200, 300]
num_layers = 1
bidirectional = True
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-4]#[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience =4

history length = 20

In [20]:
size = 20
bilstm_history_20_kfold, best_bilstm_history_20_kfold, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=None,
    data_split_seed=123,
    split_ids= torch.tensor(df_rumours['timeline_id'].astype(int)),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:37<00:00, 150.43it/s]





[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups




100%|██████████| 1/1 [01:45<00:00, 105.11s/it]

100%|██████████| 1/1 [02:26<00:00, 146.66s/it]


[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups


In [25]:
bilstm_history_20_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,k,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.5976,0.580411,"[0.6653359946773121, 0.4954864593781344]",0.208808,0.762752,0.756696,"[0.7950819672131149, 0.7183098591549295]",20,1,True,200,0.2,0.0001,0,2,False,0
0,focal,0.5928,0.578402,"[0.6563133018230924, 0.5004906771344455]",0.205242,0.766311,0.760437,"[0.7979487179487179, 0.7229254571026723]",20,1,True,200,0.2,0.0001,1,2,False,0
0,focal,0.5952,0.581269,"[0.6576454668470907, 0.5048923679060666]",0.195617,0.765125,0.759844,"[0.7954545454545456, 0.7242339832869081]",20,1,True,200,0.2,0.0001,12,2,False,0
0,focal,0.5816,0.570036,"[0.6405498281786942, 0.4995215311004785]",0.198716,0.765125,0.761279,"[0.791578947368421, 0.7309782608695652]",20,1,True,200,0.2,0.0001,123,2,False,0
0,focal,0.5584,0.548401,"[0.6155988857938719, 0.481203007518797]",0.214453,0.760380,0.757800,"[0.7827956989247312, 0.7328042328042327]",20,1,True,200,0.2,0.0001,1234,2,False,0


In [26]:
best_bilstm_history_20_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.5976,0.580411,"[0.6653359946773121, 0.4954864593781344]",0.208808,0.762752,0.756696,"[0.7950819672131149, 0.7183098591549295]",1,True,200,0.2,0.0001,0,2,False
0,focal,0.5928,0.578402,"[0.6563133018230924, 0.5004906771344455]",0.205242,0.766311,0.760437,"[0.7979487179487179, 0.7229254571026723]",1,True,200,0.2,0.0001,1,2,False
0,focal,0.5952,0.581269,"[0.6576454668470907, 0.5048923679060666]",0.195617,0.765125,0.759844,"[0.7954545454545456, 0.7242339832869081]",1,True,200,0.2,0.0001,12,2,False
0,focal,0.5816,0.570036,"[0.6405498281786942, 0.4995215311004785]",0.198716,0.765125,0.761279,"[0.791578947368421, 0.7309782608695652]",1,True,200,0.2,0.0001,123,2,False
0,focal,0.5584,0.548401,"[0.6155988857938719, 0.481203007518797]",0.214453,0.760380,0.757800,"[0.7827956989247312, 0.7328042328042327]",1,True,200,0.2,0.0001,1234,2,False


In [27]:
best_bilstm_history_20_kfold["f1"].mean()

0.5717037520357984

In [28]:
np.stack(best_bilstm_history_20_kfold["f1_scores"]).mean(axis=0)

array([0.6470887 , 0.49631881])